In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import utils

import numpy as np

from sklearn.model_selection import train_test_split

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\56236\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-aae5309234a6>", line 1, in <module>
    import tensorflow as tf
  File "c:\users\56236\appdata\local\programs\python\python37\lib\site-packages\tensorflow\__init__.py", line 101, in <module>
    from tensorflow_core import *
  File "c:\users\56236\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\__init__.py", line 40, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 959, in _find_and_load_unlocked
  File "c:\users\56236\appdata\local\programs\python\python37\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "c:\users\56236\a

KeyboardInterrupt: 

In [ ]:
num_classes = 10
img_rows, img_cols = 28, 28

In [ ]:
mnist = keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
def squash(x, axis=-1):
    squared_norm = keras.backend.sum(keras.backend.square(x), axis, keepdims=True) + keras.backend.epsilon()
    scale = keras.backend.sqrt(squared_norm)/(1+squared_norm)
    return scale*x

In [ ]:
class digitCaps(keras.layers.Layer):
    def __init__(self,in_type=1152, in_shape=8, out_type=10, out_shape=16, routings=3, activation='squash',**kwargs):
        self.in_type = in_type
        self.in_shape = in_shape       
        self.out_type = out_type
        self.out_shape = out_shape
        self.routings = routings
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = keras.layers.Activation(activation)
        super(digitCaps,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.kernel1 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel2 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel3 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel4 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel5 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel6 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel7 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel8 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel9 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel10 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        super(digitCaps,self).build(input_shape)

    def call(self,inputs):
        type1 = tf.squeeze(tf.matmul(inputs, self.kernel1),axis=2)
        type2 = tf.squeeze(tf.matmul(inputs, self.kernel2),axis=2)
        type3 = tf.squeeze(tf.matmul(inputs, self.kernel3),axis=2)
        type4 = tf.squeeze(tf.matmul(inputs, self.kernel4),axis=2)
        type5 = tf.squeeze(tf.matmul(inputs, self.kernel5),axis=2)
        type6 = tf.squeeze(tf.matmul(inputs, self.kernel6),axis=2)
        type7 = tf.squeeze(tf.matmul(inputs, self.kernel7),axis=2)
        type8 = tf.squeeze(tf.matmul(inputs, self.kernel8),axis=2)
        type9 = tf.squeeze(tf.matmul(inputs, self.kernel9),axis=2)
        type10 = tf.squeeze(tf.matmul(inputs, self.kernel10),axis=2)
        result = tf.stack([type1,type2,type3,type4,type5,type6,type7,type8,type9,type10],1)
        B = tf.zeros_like(tf.transpose(tf.transpose(result)[1]))
        for i in range(self.routings):
            C = tf.nn.softmax(B,1)
            O = tf.einsum('ijk,ijkl->ijl', C, result)
            V = squash(O,-1)
            B = B + tf.einsum('ijk,ijlk->ijl', V, result)
        return V

In [ ]:
output = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
L1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9),strides=(1,1), activation="relu")(inputs)
L2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9),strides=(2,2), activation="relu")(L1)
L3 = keras.layers.Flatten()(L2)
L4 = keras.layers.Dense(256)(L3)
L5 = keras.layers.Dense(128)(L4)
L6 = keras.layers.Dense(10)(L5)
cnn =  tf.keras.models.Model(inputs=inputs, outputs = L6)
cnn.summary()
cnn.compile(optimizer='adam',
            loss=lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)),
            metrics=['accuracy'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = digitCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
capsule =  tf.keras.models.Model(inputs=inputs, outputs = out_caps)
capsule.summary()
capsule.compile(optimizer='adam',
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1))],
            metrics=['accuracy'])

In [ ]:
x_val = x_train[:12000]
y_val = y_train[:12000]
x_train = x_train[12000:]
y_train = y_train[12000:]

In [ ]:
cnnhis = cnn.fit(x_train,y_train,epochs=20,batch_size=128,validation_data=[x_val, y_val])

In [ ]:
capsulehis = capsule.fit(x_train,y_train,epochs=20,batch_size=128,validation_data=[x_val, y_val])

In [ ]:
import matplotlib.pyplot as plt
loss = cnnhis.history['loss']
val_loss = cnnhis.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("result.png")

In [ ]:
acc = cnnhis.history['accuracy']
val_acc = cnnhis.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("result1.png")

In [ ]:
import matplotlib.pyplot as plt
loss = capsulehis.history['loss']
val_loss = capsulehis.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("result2.png")

In [ ]:
acc = capsulehis.history['accuracy']
val_acc = capsulehis.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("result3.png")

In [ ]:
num_classes = 10
img_rows, img_cols = 28, 28
mnist = keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
cnnhis = cnn.fit(x_train,y_train,epochs=3,batch_size=128)

In [ ]:
cnn.evaluate(x_test,y_test)

In [ ]:
capsulehis = capsule.fit(x_train,y_train,epochs=1,batch_size=128)

In [ ]:
capsule.evaluate(x_test,y_test)